Math
===

## 00 Find Numbers of Sole Prime Factors
### Description
From integers `n` to `m`, count the numbers that have sole prime factors $x_p$.

The original question I saw this in had specific numbers for $x_p$, but it makes more sense to me to generalize it. The original question was part of a timed test, and I freak out with timed tests and couldn't answer it, which is the only reason I include this problem here.

### Solution


Once I thought about the original problem for a few minutes after the timer expired, I came up with something similar to what is below.

In [29]:
int min = 200, max = 405, count = 0;

for (int i = min; i <= max; i++)
{
    int n = i;

    while (n % 3 == 0)
    {
        n /= 3;
    }

    if (n == 1)
    {
        count++;
        continue;
    }

    while (n % 5 == 0)
    {
        n /= 5;
    }

    if (n == 1)
    {
        count++;
    }
}

count

4

I tried to generalize the problem and then realized this would be a great problem for F#. I felt I was over complicating the problem, so I asked Bing Chat about this part of the subproblem, and then had to modify what it gave me a bit which resulted in the below function.

In [47]:
let isSolelyDivisibleByList (n:int) (list:int list) =
    let mutable x = n
    for factor in list do
        while x % factor = 0 do
            x <- x / factor
    x = 1

[15; 60] |> Seq.map (fun n -> isSolelyDivisibleByList n [3; 5]) |> Array.ofSeq

[ True, False ]

Now we have our generic test function, which we can bind to $x_p$, and then we can replace our `while` loop with a `fold` to count elements that satisfy the function.

In [54]:
let min = 200
let max = 405

let isSolelyDivisibleByListWeCareAbout n = [3;5] |> isSolelyDivisibleByList n

seq { 200 .. 2147483647 } |> Seq.fold (fun acc listItem ->
    if isSolelyDivisibleByListWeCareAbout listItem then acc + 1 else acc) 0

135

## 01 Fibonacci Sequence
### Description
A function that returns the nth value of the Fibonacci sequence.

### Solution
The standard solution is a stereotypical example to demonstrate recursion in programming languages, but it's actually a terribly unoptimized way to find the fibonacci sequence.

In [15]:
let rec fib n = 
    match n with
    | 0 -> 0
    | 1 -> 1
    | _ -> fib (n - 1) + fib (n - 2)

fib 12

144

The faster solution doesn't have much to do with programming itself but moreso math - and because of the memorization necessary, if I was asked to do this in an interview I'd probably die. However, it seems fun and potentially useful so I'd like to show an example here.

The fibonacci sequence can be defined by the following recurrence

$$f_n=f_{n-1}+f_{n-2}$$

The goal is to get the recurrence into a "generalized solution", which is a linear equation that is defined in terms of roots of $r$ of the recurrence's "characteristic equation" and the base cases of the recurrence.

To simplify that sentence, we'll break it into two parts:
1. find base cases of recurrence
2. get characteristic equation
3. get generalized solution

#### 1. Find Base Cases
This part is pretty trivial. We found the base cases in our original definition of our recurrence.

$$f_0=0$$
$$f_1=1$$

#### 2. Get Characteristic Equation
The goal is to get the recurrence into the "characteristic equation", which is a polynomial where $f_n=r^n$.

We'll now update our steps to include a substep:

1. find base cases of recurrence
2. get characteristic equation
    1. get characteristic equation
    2. get roots of $r$ of characteristic equation
3. get generalized solution in terms of $r$ and base cases

##### 2.1 Get Characteristic Equation
In a different recurrence, we'd have to account for the co-efficients and define a "recurrence relation", but the fibonacci sequence is so simple we can just substitute the symbols.

$$f_n=f_{n-1}+f_{n-2}$$
where
$$f_n=r^n$$
is

$$r^n=r^{n-1}+r^{n-2}$$

##### 2.2 Get Roots of Characteristic Equation
$$r^n=r^{n-1}+r^{n-2}$$
Goal: $r_1, r_2$

(We can view above as polynomial, e.g., multiply equation by $r^{2-n}$)
$$\frac{-b\pm\sqrt{b^2-4ac}}{2a}$$
$$\frac{-(1)\pm\sqrt{(1)^2-4(-1)(1)}}{2(-1)}$$
$$r=\frac{1\pm\sqrt{5}}{2}$$

#### 3. Get Generalized Solution

Generalized solutions are of the form depending on the number of roots of $r$:
$$f_n = A * {r_1}^n + B * {r_2}^n + C * {r_k}^n$$

In our case, we have two:
$$f_n = A * {r_1}^n + B * {r_2}^n$$

Because we have base cases of $f_n$, we can always (?) solve for our co-efficients.

$$f_0=0, f_1=1$$

Here is one such way:

$$0 = A * {\left(\frac{1 + \sqrt{5}}{2}\right)}^0 + B * {\left(\frac{1 - \sqrt{5}}{2}\right)}^0$$
$$0 = A + B$$
$$A = -B$$

$$1 = A * {\left(\frac{1 + \sqrt{5}}{2}\right)}^1 - A * {\left(\frac{1 - \sqrt{5}}{2}\right)}^1$$
$$1 = \frac{1}{2}A + \frac{\sqrt{5}}{2}A -\frac{1}{2}A + \frac{\sqrt{5}}{2}A$$
$$A = \frac{1}{\sqrt{5}}$$
$$B = -\frac{1}{\sqrt{5}}$$

I don't know why this works but it does.

$$f_n = \frac{1}{\sqrt{5}} * \left(\frac{1 + \sqrt{5}}{2}\right)^n - \frac{1}{\sqrt{5}} * \left(\frac{1 - \sqrt{5}}{2}\right)^n$$

All we have to do now is just convert this to code.

In [6]:
let sqrt5 = sqrt 5.0
let oneOverSqrt5 = 1.0 / sqrt5
let term n op = ((op 1.0 sqrt5) / 2.0) ** n

let characteristicFib n = 
    (int)(oneOverSqrt5 * term n (+) - oneOverSqrt5 * term n (-))

characteristicFib 13

233